In [1]:
import os
import glob
import pandas as pd
import numpy as np
from copy import deepcopy

In [14]:
run_directory = r'M:\gta Update\Model Outputs\2022 Pre-Valuation\gta Base Run - 2016 DT rampPen2.3,2.6'
run_folder = 'Microsim Results'
run_path = os.path.join(run_directory,run_folder)
zone_map = pd.read_csv(r'C:\Users\pechen\OneDrive - HDR, Inc\Documents - 407 Model Update\407ModelUpdate_MiscFiles_BC\407_ZoneMapping.csv', encoding='utf-8')

#### Warning Laggy - run once

In [3]:
persons_df = pd.read_csv(os.path.join(run_path,"persons.csv"))
trips_df = pd.read_csv(os.path.join(run_path,"trips.csv"))
tripmode_df = pd.read_csv(os.path.join(run_path,"trip_modes.csv"))

#### Analysis

In [49]:
trips_gta = trips_df[(trips_df.o_zone < 6000) & (trips_df.d_zone < 6000)]
trips_gta = trips_gta.merge(persons_df,on=["household_id","person_id"])
trips_gta = trips_gta[trips_gta.age >=11]
trips_gta = tripmode_df.merge(trips_gta,how = "right", on = ['household_id', 'person_id', 'trip_id'])

trips_gta["weight"] = trips_gta["weight"]/10
trips_gta["weight_z"] = trips_gta["weight"]*trips_gta["weight_x"]

trips_gta = trips_gta.merge(zone_map,left_on = "o_zone", right_on ="GTA06").drop(columns = "GTA06").rename(columns = {"region":"region_o"})
trips_gta = trips_gta.merge(zone_map,left_on = "d_zone", right_on ="GTA06").drop(columns = "GTA06").rename(columns = {"region":"region_d"})


In [90]:
save_col = ['mode', 'region_o', 'region_d', 'weight_z', 'o_depart']
auto_trips = trips_gta[trips_gta['mode']=='Auto'].drop(trips_gta.columns.difference(save_col), 1)

C:\Users\pechen\AppData\Local\Temp\ipykernel_10592\4083523077.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  auto_trips = trips_gta[trips_gta['mode']=='Auto'].drop(trips_gta.columns.difference(save_col), 1)


In [91]:
auto_trips

,mode,o_depart,weight_z,region_o,region_d
0,Auto,504.7562,1.0,1,1
11,Auto,660.0000,1.0,1,1
12,Auto,600.0000,1.0,1,1
13,Auto,705.0000,0.9,1,1
16,Auto,855.0000,0.8,1,1
...,...,...,...,...,...
22593829,Auto,365.2945,1.0,3,2
22593832,Auto,980.2874,1.0,3,2
22593833,Auto,725.1164,1.0,2,2
22593834,Auto,785.1164,1.0,2,2


In [93]:
auto_trip_summ_pm = pd.pivot_table(
    auto_trips[(trips_gta.o_depart >= 900) & (trips_gta.o_depart < 1140)],
    index = ["region_o","region_d"],
    columns = "mode", 
    values = "weight_z", 
    aggfunc = sum,
    margins = True
).drop(['All'], axis=1).drop(['All']).reset_index()

C:\Users\pechen\AppData\Local\Temp\ipykernel_10592\950475046.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  auto_trips[(trips_gta.o_depart >= 900) & (trips_gta.o_depart < 1140)],


In [96]:
auto_trip_summ_ev = pd.pivot_table(
    auto_trips[(trips_gta.o_depart >= 1140) & (trips_gta.o_depart < 1440)],
    index = ["region_o","region_d"],
    columns = "mode", 
    values = "weight_z", 
    aggfunc = sum,
    margins = True
).drop(['All'], axis=1).drop(['All']).reset_index()

C:\Users\pechen\AppData\Local\Temp\ipykernel_10592\2369628823.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  auto_trips[(trips_gta.o_depart >= 1140) & (trips_gta.o_depart < 1440)],


In [97]:
newdf2= (auto_trip_summ_ev.pivot(index='region_o',columns='region_d', values='Auto'))

c:\Users\pechen\Anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [98]:
newdf2

region_d,1,2,3,4,5,6
region_o,,,,,,
1,386514.0,25917.3,66869.7,44392.9,8941.3,1914.8
2,7290.2,105132.5,3784.0,221.5,48.3,3.0
3,39891.1,8716.8,154583.5,11239.3,841.1,192.6
4,26315.4,1137.6,9155.5,192604.9,24639.6,2634.9
5,2788.2,113.5,313.3,14644.2,97811.3,11951.3
6,430.1,30.7,84.5,1032.4,8340.3,98369.5


In [94]:
newdf= (auto_trip_summ_pm.pivot(index='region_o',columns='region_d', values='Auto'))

c:\Users\pechen\Anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [95]:
newdf

region_d,1,2,3,4,5,6
region_o,,,,,,
1,713449.5,41065.2,95764.6,55586.0,10872.6,3091.7
2,14239.6,194266.0,6668.7,446.6,127.0,29.7
3,91056.1,16404.2,302201.5,25513.6,2223.3,459.6
4,61808.2,2768.2,23958.6,387478.9,47903.0,5485.4
5,5896.5,296.3,946.6,31187.2,167659.3,27092.6
6,828.8,64.3,220.8,1621.4,15952.7,176467.4
